## Bibliotecas

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from auxiliar.arrumar_bases import *

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load(".streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df
    
vmb_path = "assets/venda-mensal-bruta_colab-bi_2025-02-11.xlsx"
agendamento_path = "assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

venda_mensal_df = pd.read_excel(vmb_path)
agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")



##Visualizar

In [ ]:

base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
periodos = base_compilada["periodo"].unique()
periodos = pd.Series(periodos)
s_date_str = "periodo: " + periodos.dt.strftime('%Y-%m-%d')


base_compilada.columns

base_comissoes = criar_comissoes(base_compilada)

base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")

base_final = juntar_bases(base_comissoes,base_avaliacoes)

In [5]:
display(base_final)

,nome_prestadora,tipo_prestadora,Unidade,comissao_total,avaliacoes_total
0,agda mikie ribeiro mikami,salão parceiro,campinas,7086.0,270.0
1,alice saccaro,prestação de serviços - biomedica,ipiranga,1136.0,120.0
2,alice saccaro,prestação de serviços - biomedica,mooca,597.0,160.0
3,aline barbosa oliveira saragoça,prestação de serviços - biomedica,santo amaro,381.0,20.0
4,amanda cristina soares da silva,salão parceiro,tijuca,5877.0,NaN
...,...,...,...,...,...
90,tassia ferreira ribeiro de oliveira,prestação de serviços - biomedica,santos,445.0,NaN
91,tatiana cristina bartolomeu angelo ricardo,prestação de serviços - biomedica,campinas,1221.0,90.0
92,tatiana cristina bartolomeu angelo ricardo,prestação de serviços - biomedica,jardins,6032.0,40.0
93,thais nayara da silveira pires,salão parceiro,londrina,9258.0,90.0


In [3]:
display(base_compilada)

,ID agendamento,ID cliente,Unidade,procedimento_padronizado,nome_prestadora,tipo_prestadora,Data,periodo,mes,valor_comissao,tipo_pagamento,receita_mes,receita_periodo,Prestador,valor_revenda
0,2581186,865755,ipiranga,radiofrequência,NaN,NaN,2025-01-04,2025-01-01,2025-01,NaN,NaN,890.0,890.0,NaN,0.00
1,2581187,865755,ipiranga,drenagem,NaN,NaN,2025-01-04,2025-01-01,2025-01,NaN,NaN,890.0,890.0,NaN,0.00
2,2598560,812996,copacabana,limpeza de pele,NaN,NaN,2025-01-09,2025-01-01,2025-01,NaN,NaN,0.0,0.0,NaN,0.00
3,2598572,812999,copacabana,drenagem,samantha batista borges nogueira,salão parceiro,2025-01-09,2025-01-01,2025-01,33.0,mensal,0.0,0.0,NaN,0.00
4,2636025,568191,jardins,co2 rosto,paula cravanzola fernandes vieira,prestação de serviços - biomedica,2025-01-04,2025-01-01,2025-01,72.0,quinzenal,0.0,0.0,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17115,orc-429506,722339,IPIRANGA,revenda,NaN,NaN,2025-01-31,2025-01-16,2025-01,NaN,NaN,0.0,0.0,Isabelle Vicheti Oliveira da Silva,2398.97
17116,orc-429522,811655,IPIRANGA,revenda,NaN,NaN,2025-01-31,2025-01-16,2025-01,NaN,NaN,0.0,0.0,Luana Ormandes Waier,96.00
17117,orc-429535,1007236,JARDINS,revenda,NaN,NaN,2025-01-31,2025-01-16,2025-01,NaN,NaN,0.0,0.0,Kathleen Bianco da Silva,96.00
17118,orc-429536,952117,JARDINS,revenda,NaN,NaN,2025-01-31,2025-01-16,2025-01,NaN,NaN,0.0,0.0,Kathleen Bianco da Silva,150.00
